## Refs

- [RDKit Documentation](https://www.rdkit.org/docs/GettingStartedInPython.html)
- [PubChem](https://pubchem.ncbi.nlm.nih.gov/) - find SDF files for molecules
- [YouTube Tutorial](https://www.youtube.com/watch?v=NozaWUkJ3YM)


## Tools Base


In [16]:
from rdkit.Chem import SDMolSupplier, Mol, Atom
from rdkit.Chem import Draw
from src import SDF_DIR


def load_sdf(file: str) -> list[Mol]:
    """Loads Molecule from file to python object (Properties in the SDF file are used to set properties on each molecule)"""
    sdf_path = f"{SDF_DIR.joinpath(file)}"
    mols: list[Mol] = [x for x in SDMolSupplier(sdf_path)]

    assert mols, "No molecules loaded. Check if the SDF file is empty or malformed."
    assert all(mol is not None for mol in mols), (
        f"Some molecules in the file '{sdf_path}' failed to parse. "
        "Check for syntax errors or unsupported atom types."
    )

    return mols

## RDkit magnetic experiments
`Preliminary coding & studying RDKit`


In [ ]:
from rdkit.Chem import rdMolDescriptors
from rdkit import Chem

for i, mol in enumerate(load_sdf(file="As-benzene-arsenic acid.sdf")):
    print(f"I am molecule {i + 1}")

    rdMolDescriptors.CalcOxidationNumbers(mol)

    mol = Chem.AddHs(mol)
    atoms: list[Chem.Atom] = [
        a for a in mol.GetAtoms()
    ]  # Creates a list of atoms for each molecule

    # Assign charge taken from SDF file ONLY for atoms without covalent bonds
    for atom in atoms:
        covalent_bond = (
            atom.GetTotalDegree()
        )  # Number of covalent bonds including implicit Hs
        if covalent_bond == 0:
            atom_charge = atom.GetFormalCharge()
        else:
            atom_charge = None

    ox_state_atoms = [  # TODO:: Should this be any input from the user?
        "As",
        "Hg",
        "Pb",
    ]

    for atom in atoms:
        if (
            atom.HasProp("OxidationNumber")
            and atom_charge is None
            and atom.GetSymbol() in ox_state_atoms
        ):
            ox_state = atom.GetProp("OxidationNumber")
        else:
            ox_state = None

        neighbors = [nbr.GetSymbol() for nbr in atom.GetNeighbors()]
        neighbor_symbols = ", ".join(neighbors) if neighbors else "none"

    ring_atoms = [  ## TODO:: Should this be any input from the user?
        "N",
        "C",
    ]

    for atom in atoms:
        if atom.GetSymbol() in ring_atoms:
            in_ring = atom.IsInRing()
        else:
            in_ring = None

        print(
            f"""
            Atom: {atom.GetSymbol()},  
            IsInRing: {atom.IsInRing()},
            in_ring: {in_ring},
            GetNoBondedAtoms: {atom.GetTotalDegree()},
            FormalCharge: {atom.GetFormalCharge()},
            atom_charge: {atom_charge}
            ox_state: {ox_state}, 
            id: {atom.GetIdx()}, 
            neighbors: {neighbor_symbols}, 
            """
        )

I am molecule 1

            Atom: C,  
            IsInRing: True,
            in_ring: True,
            GetNoBondedAtoms: 3,
            FormalCharge: 0,
            atom_charge: None
            ox_state: None, 
            id: 0, 
            neighbors: O, 
            

            Atom: C,  
            IsInRing: True,
            in_ring: True,
            GetNoBondedAtoms: 3,
            FormalCharge: 0,
            atom_charge: None
            ox_state: None, 
            id: 1, 
            neighbors: O, 
            

            Atom: As,  
            IsInRing: True,
            in_ring: None,
            GetNoBondedAtoms: 2,
            FormalCharge: 0,
            atom_charge: None
            ox_state: None, 
            id: 2, 
            neighbors: O, 
            

            Atom: C,  
            IsInRing: True,
            in_ring: True,
            GetNoBondedAtoms: 3,
            FormalCharge: 0,
            atom_charge: None
            ox_state: None, 
   

In [ ]:
from constants import PASCAL_CONST

sum_dia_contr = 0

for i in range(len(mols)):
    mol = mols[i]

    print(f"I am molecule {i + 1}")

    rdMolDescriptors.CalcOxidationNumbers(mol)

    mol = Chem.AddHs(mol)
    atoms: list[Chem.Atom] = [
        a for a in mol.GetAtoms()
    ]  # Creates a list of atoms for each molecule

    for atom in atoms:
        symbol = atom.GetSymbol()
        if symbol in PASCAL_CONST:
            # retrieve const data
            covalent_data = PASCAL_CONST[symbol]["covalent"]
            ionic_data = PASCAL_CONST[symbol]["ionic"]["charge"]
            ox_state_data = PASCAL_CONST[symbol]["covalent"]["ox_state"]

            if in_ring:
                sum_dia_contr += covalent_data["ring"]

            """TODO: Resolve the error"""
            if not in_ring:
                sum_dia_contr += covalent_data["open_chain"]

            """TODO: Fix incompatible format of oxidation state. Ox_state from RDKit: int, Ox_state in PASCAL_CONST: string."""
            # if ox_state is not None:
            # ....

            """TODO: Fix incompatible format of charge. Charge from RDKit: int; Charge in PASCAL_CONST: string."""
            # if atom_charge is not None:
            # ....
    print(sum_dia_contr)

I am molecule 1


TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'